<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/12_gan/dcgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Generative Adversarial Networks (DC-GAN)
## 目的
DC-GANによって画像の生成をして動作を理解する．

## モジュールのインポート

はじめに必要となるモジュールをインポートします．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算を行うことが可能です．
Falseとなっている場合は，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．

In [ ]:
import os
import zipfile
import urllib.request
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## ネットワークの構築
オリジナルのGANは，全結合層とReLUのみで構築される非常にシンプルなネットワークでした．
シンプルな反面，学習が不安定になったり，高解像な画像生成は難しいという問題がありました．

Deep Convolutional GAN (DC-GAN)は，名前の通り畳み込み処理を用いたネットワークの構築することによって，オリジナルのGANより綺麗な画像を生成することを可能としています．

また，DC-GANは，以下に示す点でもオリジナルのGANと異なります．
* Batch Normalizationを利用する．（Discriminatorの1層目と最終層は除く）
* Discriminatorの活性化関数は，Leaky ReLUを使用する．
* Generatorの最終層はtanh関数を使用する．（それ以外の層は全てReLU）
* Deconvolutionの使用，PoolingではなくConvolutionによるDownsampling

これらを考慮してDiscriminatorとGeneratorのネットワークを構築します．
このとき，入力引数として`img_size`を指定します．
これは生成する画像サイズを指定する引数です．
この値に応じて，畳み込みの数を適応的に設定し，ネットワークを構築します．
GeneratorとDiscriminator共に重みパラメータは平均が0，偏差が0.02の正規分布で，バイアスは0で初期化します．

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim=100, out_ch=3, img_size=32):
        super(Generator, self).__init__()
        n_layer = int(np.log2(img_size)) - 2
        self.model = nn.Sequential()

        features = 512
        self.model.add_module('Deconv_first', nn.ConvTranspose2d(latent_dim, features, 2, 2, bias=True))
        self.model.add_module('BN_first', nn.BatchNorm2d(features))
        self.model.add_module('Act_first', nn.ReLU(inplace=True))

        in_features = features
        for i in range(n_layer):
            out_features = in_features // 2
            self.model.add_module('Deconv%d' % i, nn.ConvTranspose2d(in_features, out_features, 4, 2, 1, bias=True))
            self.model.add_module('BN%d' % i, nn.BatchNorm2d(out_features))
            self.model.add_module('Act%d' % i, nn.ReLU(inplace=True))
            in_features = out_features

        self.model.add_module('Deconv_last', nn.ConvTranspose2d(in_features, out_ch, 2, 2, bias=True))
        self.model.add_module('Act_last', nn.Tanh())
        self.weight_init()

    def weight_init(self):
        for m in self.model:
          if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0.0, 0.02)
            m.bias.data.zero_()

    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self, in_ch=3, img_size=32):
        super(Discriminator, self).__init__()
        n_layer = int(np.log2(img_size)) - 2
        self.model = nn.Sequential()

        features = 32
        self.model.add_module('Conv_first', nn.Conv2d(in_ch, features, 4, 2, 1, bias=True))
        self.model.add_module('Act_first', nn.LeakyReLU(negative_slope=0.2))

        in_features = features
        for i in range(n_layer):
            out_features = in_features * 2
            self.model.add_module('Conv%d' % i, nn.Conv2d(in_features, out_features, 4, 2, 1, bias=True))
            self.model.add_module('BN%d' % i, nn.BatchNorm2d(out_features))
            self.model.add_module('Act%d' % i, nn.LeakyReLU(negative_slope=0.2))
            in_features = out_features

        self.model.add_module('Conv_last', nn.Conv2d(in_features, 1, 2, 1, bias=True))
        self.weight_init()

    def weight_init(self):
        for m in self.model:
          if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0.0, 0.02)
            m.bias.data.zero_()

    def forward(self, x):
        return self.model(x).view(-1)

## データセットと最適化関数
データセットにはMNISTを使用します．
最適化関数はAdam optimizer使用し，学習率$2\times 10^4$，betaの値を$0.5, 0.999$として学習します．

最後に，構築したネットワークモデルの詳細を表示します．
指定した`img_size`の値に応じて畳み込みの数が変わります．

In [ ]:
# データセットの設定
transform_training = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()])
mnist_data = datasets.MNIST(root='./data', train=True, transform=transform_training, download=True)
training_data = DataLoader(mnist_data, batch_size=128, shuffle=True)


latent_dim = 100
G = Generator(latent_dim=latent_dim, out_ch=1, img_size=32)
D = Discriminator(in_ch=1, img_size=32)
if use_cuda:
    G = G.cuda()
    D = D.cuda()
opt_g = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_d = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
print('=' * 50)
print(G)
print('=' * 50)
print(D)
print('=' * 50)

## GANの学習

GANの学習を行います．
学習のプロセスは，オリジナルのGANと同じため，説明は割愛します．

※ **こDC-GANはオリジナルのGANよりも演算量が増えるため学習に時間がかかります．演習では学習を割愛し，学習済みモデルを用いて画像の生成結果を確認します．**
ご興味のある方は，講義終了後にご自身でうごかしてみてください．

In [ ]:
n_epoch = 20
n_critic = 1

criterion = nn.BCEWithLogitsLoss()

for epoch in range(1, n_epoch+1):
    Tensor = torch.cuda.FloatTensor
    G.train()
    D.train()
    for idx, (real_x, _) in enumerate(training_data):
        if use_cuda:
            real_x = real_x.cuda()
        batch = real_x.size(0)
        flag_real = Tensor(batch).fill_(1.0)
        flag_fake = Tensor(batch).fill_(0.0)

        for _ in range(n_critic):
            D.zero_grad()
            z = torch.randn(batch, latent_dim, 1, 1)
            if use_cuda:
                z = z.cuda()
            fake_x = G(z)
            out_real = D(real_x)
            out_fake = D(fake_x.detach())
            loss_real = criterion(out_real, flag_real)
            loss_fake = criterion(out_fake, flag_fake)
            dis_loss = loss_real + loss_fake
            dis_loss.backward()
            opt_d.step()

        G.zero_grad()
        z = torch.randn(batch, latent_dim, 1, 1)
        if use_cuda:
            z = z.cuda()
        fake_x = G(z)
        out_gen = D(fake_x)
        gen_loss = criterion(out_gen, flag_real)
        gen_loss.backward()
        opt_g.step()

        if idx % 100 == 0:
            print('Training epoch: {} [{}/{} ({:.0f}%)] | D loss: {:.6f} | G loss: {:.6f} |'\
                  .format(epoch, idx * len(real_x), len(training_data.dataset),
                  100. * idx / len(training_data), dis_loss.item(), gen_loss.item()))

    torch.save(G.state_dict(), "G_%04d.pt" % epoch)
    torch.save(D.state_dict(), "D_%04d.pt" % epoch)

## 学習したGeneratorによる画像生成

学習したGeneratorを用いて画像を生成します．

まず，学習済みモデルのパラメータを読み込みます．




ここでは，正規分布に従い乱数`z`を生成し，それをGeneratorへと入力することで，画像生成をおこない，その結果を表示します．

適切な学習が行われている場合は，0~9の数字の画像が生成されます．

In [ ]:
# データのダウンロード
import gdown
gdown.download('https://drive.google.com/uc?id=1EUxuBqXVzm18rwlbU_FHk7NtYY9rc9Y0', 'dcgan_model.zip', quiet=True)
!unzip -q dcgan_model.zip


# 学習済みモデルの読み込み
G.load_state_dict(torch.load("dcgan_model/G_0100.pt"))
D.load_state_dict(torch.load("dcgan_model/D_0100.pt"))

In [ ]:
z = torch.randn(100, 100, 1, 1)
if use_cuda:
    z = z.cuda()

G.eval()
with torch.no_grad():
    test_img = G(z)

if use_cuda:
    test_img = test_img.cpu()
_test_img = (test_img * 256.).clamp(min=0., max=255.).squeeze().data.numpy()

fig = plt.figure(figsize=(10, 10))
for i, im in enumerate(_test_img):
    ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
    ax.imshow(im, 'gray')

## 課題
1. データセットをMNISTからCIFAR-10に変更して学習してみましょう．
2. 2つの潜在変数の間を補間するように画像生成した場合，どのような画像が生成されるか確認してください．

# 参考文献
[1] Alec Radford, Luke Metz and Soumith Chintala, Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks, ICLR, 2016.